<a href="https://colab.research.google.com/github/mad1122/DA1/blob/main/school_data_delhi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PROJECT: SCHOOL DATA

### Createing a dataset by scraping data from school database website: https://school.careers360.com/schools/schools-in-delhi
### Analayse the data gathered after scraping it from different pages within the website
### Denoting the key attributes of different schools 

# Importing needed libraries:


In [1]:
import numpy as np
import pandas as pd
import random
from numpy.random import randn
import os
import seaborn as sn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import pytz
import re
from scipy.stats import stats
import requests
from bs4 import BeautifulSoup as bs


**Using requests to download webpage with data**

In [10]:
page = 1
sch_nam=[]
sch_url=[]
sch_rat=[]
while page != 75:
  d_d_url=f'https://school.careers360.com/schools/schools-in-delhi?page={page}'
  distt= requests.get(d_d_url)
  doc1= bs(distt.text, 'html.parser')

  tags_1= doc1.find_all('h3', {'class' : 'blockHeading'})
  tags_2= doc1.find_all('div', {'class' : 'rating1'})
  
  for nam in tags_1:
    sch_nam.append(nam.text)
  
  for url in tags_1:
    sch_url.append(url.find('a')['href'])
  
  for rat in tags_2:
    pattern = "strong>(.*?)</"
    rating_tag = re.search(pattern, str(rat)).group(1)
    sch_rat.append(rating_tag)
  
  page = page + 1

sch_dict={'Name_location':sch_nam, 'Rating':sch_rat, 'URL':sch_url}
df1=pd.DataFrame.from_dict(sch_dict,orient='index')
df1 = df1.transpose()




      
      


In [103]:
sch_nam=[]
sch_fee=[]
sch_bd=[]
sch_own=[]
sch_afn=[]
sch_gnd=[]
sch_siz=[]
sch_lan=[]
sch_tch=[]
sch_clss=[]
sch_enrol=[]


for i in sch_url[:2200]:
  school= requests.get(i)
  doc2= bs(school.text, 'html.parser')

  ver1 = doc2.find_all('h1')
  ver2=ver1[0].text.strip()
  ver3=ver2.split(',')[0]
  sch_nam.append(ver3)

  fee_tag=doc2.find('span', {'class' : 'feeTxt'})
  if fee_tag is None:
    pass
  else:
    a=fee_tag.text
    fee=a[a.find('\n')+len('\n'):a.rfind('/-')]
    sch_fee.append(fee)

  data2 = []
  table2 = doc2.find('div', attrs={'id':'information'})
  rows2 = table2.find_all('tr')
  for row in rows2:
    
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data2.append([ele for ele in cols if ele])
  dff=pd.DataFrame(data2)
  dff=dff.T
  dff.columns=dff.iloc[0]
  dff=dff.drop(dff.index[0])
  if 'Board' in dff.columns:
    sch_bd.append(dff['Board'].to_string(index=False))
  else:
    sch_bd.append(None)
    pass
  if 'Ownership' in dff.columns:
    sch_own.append(dff['Ownership'].to_string(index=False))
  else:
    sch_own.append(None)
    pass
  if 'Affiliation Number' in dff.columns:
    sch_afn.append(dff['Affiliation Number'].to_string(index=False))
  else:
    sch_afn.append(None)
    pass
  if 'Gender' in dff.columns:
    sch_gnd.append(dff['Gender'].to_string(index=False))
  else:
    sch_gnd.append(None)
    pass
  if 'Campus Size' in dff.columns:
    sch_siz.append(dff['Campus Size'].to_string(index=False))
  else:
    sch_siz.append(None)
    pass

  data3 = []
  table1 = doc2.find('div', attrs={'class':'enroll_block'})
  if table1 is None:
    pass
  else:
    table_body1 = table1.find('tbody')
    rows1 = table_body1.find_all('tr')
    for row in rows1:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data3.append([ele for ele in cols if ele])
    if data3[1][1][0]is'N':
      pass
    else:
      dff1=pd.DataFrame(data3)
      dff1=dff1.drop(dff1.index[0])
      dff1[1]=dff1[1].astype('float')
      dff1['total']=dff1[1].sum()
      sch_enrol.append(dff1['total'][1])

  data4 = []
  a = doc2.find('div', attrs={'id':'academics'})
  if a is None:
    pass
  else:
    rows3 = a.find_all('tr')
    for row in rows3:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data4.append([ele for ele in cols if ele])
    dff2=pd.DataFrame(data4)
    dff2=dff2.T
    dff2.columns=dff2.iloc[0]
    dff2=dff2.drop(dff2.index[0])
    if 'Medium(s) of Instruction' in dff2.columns:
      sch_lan.append(dff2['Medium(s) of Instruction'].to_string(index=False))
    else:
      sch_lan.append(None)
      pass
    if 'No. of Teachers' in dff2.columns:
      sch_tch.append(dff2['No. of Teachers'].to_string(index=False))
    else:
      sch_tch.append(None)
      pass
    if 'No. of Classrooms' in dff2.columns:
      sch_clss.append(dff2['No. of Classrooms'].to_string(index=False))
    else:
      sch_clss.append(None)
      pass

sch_dict2={'Affiliation_number':sch_afn, 'Name':sch_nam,'Fee':sch_fee, 'Board':sch_bd, 'Owned_by':sch_own, 
           'Gender':sch_gnd, 'Campu_size':sch_siz, 'Enrollments':sch_enrol, 'No_of_teachers':sch_tch,
           'No_of_classrooms':sch_clss,'Language_of_instruction':sch_lan}
dfs=pd.DataFrame.from_dict(sch_dict2,orient='index')
dfs = dfs.T

In [104]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Affiliation_number       2198 non-null   object
 1   Name                     2200 non-null   object
 2   Fee                      306 non-null    object
 3   Board                    2193 non-null   object
 4   Owned_by                 2185 non-null   object
 5   Gender                   2157 non-null   object
 6   Campu_size               1908 non-null   object
 7   Enrollments              467 non-null    object
 8   No_of_teachers           645 non-null    object
 9   No_of_classrooms         468 non-null    object
 10  Language_of_instruction  2110 non-null   object
dtypes: object(11)
memory usage: 189.2+ KB


# New Section

In [ ]:
'''df2.info() #/content/Delhi_school_data1.csv
df3.info() #/content/Delhi_school_data2.csv
df4=pd.read_csv('Delhi_school_data4.csv',index_col=None)
df5.info() #/content/Delhi_school_data5.csv
df6=pd.read_csv('Delhi_school_data6.csv',index_col=None)
df7=pd.read_csv('Delhi_school_data3.csv',index_col=None)
dsf=pd.concat([ds1,ds2,ds3,ds4,ds5,ds6])
dsf.info()'''


In [ ]:
'''dsf1=pd.merge(df1, dfs, right_index=True, left_index=True)
dsf1.info()'''

In [107]:
#dsf1.to_csv('Delhi_schools.csv', index=False)

In [ ]:
'''df1.info()'''

In [ ]:
'''df_sch_del_bas=df1
df_sch_del_bas.describe()'''

In [54]:
#df_sch_del_bas.to_csv('Delhi_school_basic.csv')

In [ ]:
'''sch_url[2199]'''

In [ ]:
'''
Delhi_Districts_url='https://school.careers360.com/schools/amrita-vidyalayam-pushp-vihar-delhi'
distt= requests.get(Delhi_Districts_url)
del_dist=distt.text
doc5= bs(del_dist, 'html.parser')'''

In [ ]:
#pattern = "strong>(.*?)</"
#rating_tag = re.search(pattern, str(tags_2[1])).group(1)
'''
sch_rat=[]
for rat in tags_2:
  pattern = "strong>(.*?)</"
  rating_tag = re.search(pattern, str(rat)).group(1)
  sch_rat.append(rating_tag)
sch_rat[5:15]'''

In [ ]:
'''tags_1[0].text

sch_nam=[]
for nam in tags_1:
  sch_nam.append(nam.text)
sch_nam[5:12]'''

In [ ]:
'''sch_dict={'Name':sch_nam, 'Rating':sch_rat, 'URL':sch_url}
df1=pd.DataFrame(sch_dict)
'''


In [ ]:
'''ver1 = doc2.find_all('h1')
ver2=ver1[0].text.strip()
ver2.split(',')[0]'''


In [65]:
'''df22= df2.loc[df2['Enrollments'].notnull()]
df_sch_del=df2'''